# Affine Model with Stochastic Market Price of Risk

The model uses as state variables, 
1. the short-rate $r_t$
2. its own stochastic reversion level $\theta_t$
3. the market-price-of-risk $\lambda_t$

The SDEs are as follows, 

$$
\begin{aligned}
dr_t &= \kappa_{r}\left[\theta_t - r_t\right]dt + \sigma_r dz_t^r\\
d\theta_t &= \kappa_{\theta}\left[\theta^{\infty} - \theta_t\right]dt + \lambda_t^{\theta}\sigma_{\theta}dt+ \sigma_{\theta} dz_t^r\\
d\lambda_t^{\theta} &= \kappa_{\lambda}\left[\lambda^{\infty} - \lambda^{\theta}_t\right]dt + \sigma_{\lambda} dz_t^{\lambda}\\
\end{aligned}
$$
with
$$
\begin{aligned}
E\left[drd\theta\right]&=\rho_{r\theta}\\
E\left[drd\lambda\right]&=\rho_{r\lambda}\\
E\left[d\theta d\lambda\right]&=\rho_{\theta \lambda}\\
\end{aligned}
$$

The short-rate also takes an affine form, 

$$
\begin{aligned}
r_t &= u_r + \mathbf{g}^T \mathbf{x}_t \\
\text{with}\\
u_r &= 0\\
\mathbf{g}^T &= \begin{bmatrix} 0 & 0 & 1 \end{bmatrix} 

\end{aligned}
$$

The state variables (in vector form), 

$$
\mathbf{x}_t=\begin{bmatrix} x_t^1 \cr x_t^2 \cr x_t^3 \end{bmatrix} = \begin{bmatrix} \lambda_t^{\theta} \cr \theta_t \cr r_t \end{bmatrix} 
$$

follows the dynamics (in vector form, 

$$
\begin{aligned}
d\mathbf{x}_t &= \mathcal{K}(\pmb{\theta} - \mathbf{x}_t)dt + Sd\mathbf{z}_t\\
d\mathbf{x}_t&=\begin{bmatrix} dx_t^1 \cr dx_t^2 \cr dx_t^3 \end{bmatrix} = \begin{bmatrix} \kappa_{\lambda} & 0 & 0\\ -\sigma_{\theta} & \kappa_{\theta} & 0 \\ 0 & -\kappa_r & \kappa_r\end{bmatrix} \left(\begin{bmatrix} \lambda^\infty\\ \Theta^\infty +\frac{\sigma_\theta\lambda^\infty}{\kappa_\theta} \\  \Theta^\infty +\frac{\sigma_\theta\lambda^\infty}{\kappa_\theta}\end{bmatrix}- \begin{bmatrix} \lambda_t^{\theta} \cr \theta_t \cr r_t \end{bmatrix} \right)dt +\begin{bmatrix} s_{11} & 0 & 0\\ s_{21} & s_{22} & 0 \\ s_{31} & s_{32} & s_{33} \end{bmatrix} d\mathbf{z}_t\\
\text{where}\\
s_{11}&=\sigma_\lambda\\
s_{21}&=\sigma_{\theta}\rho_{\theta \lambda}\\
s_{22}&=\sigma_{\theta}\sqrt{1-\rho^2_{\theta \lambda}}\\
s_{31}&=\sigma_{r}\rho_{ \lambda r}\\
s_{32}&=\sigma_r\frac{\rho_{\theta r}- \rho_{\lambda\theta}\rho_{\lambda r}}{\sqrt{1-\rho^2_{\theta \lambda}}}\\
s_{33}&=\sigma_r \sqrt{1-\rho^2_{\lambda r}-\frac{\left(\rho_{\theta r}- \rho_{\lambda\theta}\rho_{\lambda r}\right)^2}{1-\rho^2_{\theta \lambda}}}
\end{aligned}
$$



Altogether in one, 

$$
d\mathbf{x}_t=\begin{bmatrix} dx_t^1 \cr dx_t^2 \cr dx_t^3 \end{bmatrix} = \begin{bmatrix} \kappa_{\lambda} & 0 & 0\\ -\sigma_{\theta} & \kappa_{\theta} & 0 \\ 0 & -\kappa_r & \kappa_r\end{bmatrix} \left(\begin{bmatrix} \lambda^\infty\\ \Theta^\infty +\frac{\sigma_\theta\lambda^\infty}{\kappa_\theta} \\  \Theta^\infty +\frac{\sigma_\theta\lambda^\infty}{\kappa_\theta}\end{bmatrix}- \begin{bmatrix} \lambda_t^{\theta} \cr \theta_t \cr r_t \end{bmatrix} \right)dt +\begin{bmatrix} \sigma_\lambda & 0 & 0\\ \sigma_{\theta}\rho_{\theta \lambda} & \sigma_{\theta}\sqrt{1-\rho^2_{\theta \lambda}} & 0 \\ \sigma_{r}\rho_{ \lambda r} & \sigma_r\frac{\rho_{\theta r}- \rho_{\lambda\theta}\rho_{\lambda r}}{\sqrt{1-\rho^2_{\theta \lambda}}} & \sigma_r \sqrt{1-\rho^2_{\lambda r}-\frac{\left(\rho_{\theta r}- \rho_{\lambda\theta}\rho_{\lambda r}\right)^2}{1-\rho^2_{\theta \lambda}}} \end{bmatrix} d\mathbf{z}_t\\
$$

The bond price $P_t^T$ is given by, 
$$
P_t^T = e^{A_t^T+\mathbf{B}^T\mathbf{x}_t}
$$

$$
\Theta
$$

In [20]:
from collections import namedtuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
from numpy import linalg as LA
from pandas.tseries.offsets import MonthEnd
from scipy.linalg import expm


In [52]:
PCAffineParams = namedtuple('PCAffineParams', '''
r0
theta_0
lambda_0
mu_r
g_T
kappa_r
kappa_theta
kappa_lambda
sigma_r
sigma_theta
sigma_lambda
theta_hat_t
lambda_hat_t
rho_r_theta
rho_r_lambda
rho_theta_lambda
'''
)
    

In [15]:
maturities = [2, 5, 10]

In [133]:
yields_data = pd.read_excel('feds200628.xlsx', sheet_name='Yields', skiprows=1, index_col='Date', parse_dates=True )
yields_data = yields_data.div(100.)
# yields_data = yields_data.loc[:, [f'SVENY{x:02d}' for x in range(1, 31)] ]
# yields_data = yields_data.loc[:, [f'SVENY{x:02d}' for x in maturities] ]
yields_data = yields_data.dropna(how='any')

In [10]:
#yields_data.plot(figsize=(12, 8))

In [11]:
#yields_data

In [12]:
#yields_data.mean()

In [13]:
#yields_data - yields_data.mean()

In [15]:
yield_changes = yields_data.diff()
# yield_changes

In [26]:
yields_cov = yield_changes.cov()
yields_cov

,SVENY02,SVENY05,SVENY10
SVENY02,4.636868e-07,4.145176e-07,3.274264e-07
SVENY05,4.145176e-07,4.616632e-07,4.019395e-07
SVENY10,3.274264e-07,4.019395e-07,4.219698e-07


In [28]:
w, v = LA.eig(yields_cov)
w, v

(array([1.21398334e-06, 1.15291965e-07, 1.80445201e-08]),
 array([[-0.57490852, -0.7196263 ,  0.38938179],
        [-0.60881959,  0.05829518, -0.79116394],
        [-0.54664329,  0.69191015,  0.47163699]]))

In [31]:
np.matmul(v, v.T)

array([[ 1.00000000e+00,  5.55111512e-17, -1.11022302e-16],
       [ 5.55111512e-17,  1.00000000e+00,  0.00000000e+00],
       [-1.11022302e-16,  0.00000000e+00,  1.00000000e+00]])

In [16]:
eigenvalues = [0.01,0.5,0.6]
eigenmatrix = np.diag(eigenvalues)
eigenmatrix

array([[0.01, 0.  , 0.  ],
       [0.  , 0.5 , 0.  ],
       [0.  , 0.  , 0.6 ]])

In [25]:
( np.array(eigenvalues) * 7 ) / np.array([4.,5.,6.])

array([0.0175, 0.7   , 0.7   ])

In [26]:
 np.array([1.,2.,3.]) / np.array([4.,5.,6.])

array([0.25, 0.4 , 0.5 ])

In [8]:
np.array([1,2,3]).reshape((3, 1))

array([[1],
       [2],
       [3]])

In [ ]:
# kappa_r, kappa_theta, kappa_lambda, 
#                     sigma_r, sigma_theta, sigma_lambda, 
#                     rho_lambda_theta, rho_lambda_r, rho_theta_r,
#                     lambda_lt, theta_lt, T):

In [130]:
class PCAffineModel:

    def __init__(self, yields_data):
        self.yields_data = yields_data


    def fit(self, initial_params=None):

        # 1. calibrate to the covariance matrix
        yield_changes = self.yields_data.diff()
        yields_cov = yield_changes.cov()
        print(yields_cov)


        # eigen_yield_cov, omega_mat = LA.eig(yields_cov)
        # self.KappaMatrix = self.makeKappaMatrix(eigenvalues, maturities, omega_mat)
        # # self.FMatrix = self.makeFMatrix(eigenvalues, maturities)
        # self.makeAMatrix(eigenvalues, maturities, self.mu_r)

    def makeFMatrix(self, eigenvalues, MMatrix, T):
        Fmatrix = np.zeros((len(eigenvalues), len(eigenvalues)))

        for i, mat in enumerate(eigenvalues):
            for j, eig in enumerate(eigenvalues):
                li = eigenvalues[i]
                lj = eigenvalues[j]
                Fmatrix[i, j] = MMatrix[i, j] * (( 1 - np.exp((li + lj)*T) )/ (li+lj))
        return Fmatrix
    
    def makeKappaMatrix(self, kappa_lambda, sigma_theta, kappa_theta, kappa_r ):

        return np.array([
            [kappa_lambda, 0., 0.], 
            [-sigma_theta, kappa_theta, 0.], 
            [0., -kappa_r, kappa_r], 
        ])

    def makeThetaVec(self, params):
        p = params
        # lambda_lt is lambda_infty
        # lambda_lt, theta_lt, sigma_theta, kappa_theta
        return np.array([p.lambda_hat_t, p.theta_hat_t + (p.sigma_theta * p.lambda_hat_t)/p.kappa_theta,  
                         p.theta_hat_t + (p.sigma_theta * p.lambda_hat_t)/p.kappa_theta])

    def makeSMatrix(self, params): # sigma_lambda, sigma_theta, sigma_r, rho_lambda_theta, rho_lambda_r, rho_theta_r)
        p = params
        return np.array([
            [p.sigma_lambda, 0., 0.], 
            [p.sigma_theta * p.rho_theta_lambda, p.sigma_theta * np.sqrt(1-p.rho_theta_lambda**2), 0.], 
            [p.sigma_r*p.rho_r_lambda, p.sigma_r*((p.rho_r_theta-p.rho_theta_lambda*p.rho_r_lambda)/(np.sqrt(1-p.rho_theta_lambda**2))), 
                p.sigma_r*np.sqrt(1 - p.rho_r_lambda**2 - (((p.rho_r_theta-p.rho_theta_lambda*p.rho_r_lambda)**2)/(1-p.rho_theta_lambda**2)))], 
            ])

    def makeBRowVec(self, eigenvalues, maturities, omega):
        res = np.zeros((1,3))
        kappa_matrix = self.makeKappaMatrix(eigenvalues, maturities, omega)
        kappa_matrix_inv = LA.inv(kappa_matrix)

        FMatrix = self.makeFMatrix(eigenvalues, maturities)
        g = omega.T @ ( LA.inv(FMatrix.T).dot(np.ones(3)) )
        for i, tau in enumerate(maturities):
            res[i] = g.T @ ( kappa_matrix_inv @ ( expm(-kappa_matrix * tau) - np.eye(3) ) )
        return res
    
    def makeDTMatrix(self, eigenvalues, T ):

        return np.diag( (1-np.exp(np.array(eigenvalues) * T)) / np.array(eigenvalues) )

    def makeMMatrix(self, eigenvec_inv, SMatrix):

        C = SMatrix @ SMatrix.T
        return ( eigenvec_inv @ ( C @ eigenvec_inv.T) )

    def makeAMatrix(self, params, T):
        p = params
        Int1 = -p.mu_r * T
        
        #Int2
        kappaMatrix = self.makeKappaMatrix( p.kappa_lambda, p.sigma_theta, p.kappa_theta, p.kappa_r )
        kappaMatrix_inv = LA.inv(kappaMatrix)
        eigenvalues, eigenvectors = LA.eig(kappaMatrix)
        eigenD = np.diag(eigenvalues)
        eigenD_inv = LA.inv(eigenD)
        eigenvec_inv = LA.inv(eigenvectors)

        Dmatrix = self.makeDTMatrix(eigenvalues, T)
        Theta_Vec = self.makeThetaVec(params)
        Int2 = p.g_T @ ( ( eigenvectors @ ( eigenD_inv @ ( Dmatrix @ (eigenD @ (eigenvec_inv @ Theta_Vec) ) ) ) ) - (Theta_Vec * T) )

        #Int3a
        SMatrix = self.makeSMatrix(params)
        MMatrix = self.makeMMatrix(eigenvec_inv, SMatrix)
        FMatrix = self.makeFMatrix(eigenvalues, MMatrix, T)
        Int3a = 0.5 * p.g_T @ (kappaMatrix_inv @ ( eigenvectors @ ( FMatrix @ (eigenD_inv @ (eigenvectors.T @ p.g_T.T) ) ) ) )

        #Int3b
        C = SMatrix @ SMatrix.T
        Int3b = -0.5 * p.g_T  @ ( kappaMatrix_inv @ ( C @ ( eigenvec_inv.T @ ( Dmatrix @ (eigenD_inv @ (eigenvectors.T @ p.g_T.T) ) ) ) ) )

        #Int3c
        Int3c = -0.5 * p.g_T @ ( kappaMatrix_inv @ ( eigenvectors @ ( Dmatrix @ ( eigenvec_inv @ ( C @ ( LA.inv(kappaMatrix.T) @ p.g_T.T ) ) ) ) ) )

        #Int3d
        Int3d = 0.5 * p.g_T @ ( kappaMatrix_inv @ ( C @ ( LA.inv(kappaMatrix.T) @ p.g_T.T ) ) ) * T
        return Int1 + Int2 + Int3a + Int3b + Int3c + Int3d

    def makeBMatrix(self, params, T):
        p = params
        kappaMatrix = self.makeKappaMatrix( p.kappa_lambda, p.sigma_theta, p.kappa_theta, p.kappa_r )

        return p.g_T @ ( LA.inv(kappaMatrix) @ (expm(-kappaMatrix * T) - np.eye(3)) )
        

In [131]:
params = PCAffineParams(        
    r0 = -0.0042,
    theta_0 = 0.0344,
    lambda_0 = 0.0744,
    mu_r = 0.,
    g_T = np.array([0.,0.,1.]),
    kappa_r = 0.3437,
    kappa_theta = 0.0085,
    kappa_lambda = 0.2816,
    sigma_r = 0.0050,
    sigma_theta = 0.0157,
    sigma_lambda = 0.12,
    theta_hat_t = 0.035,
    lambda_hat_t = 0.1287,
    rho_r_theta = 0.6,
    rho_r_lambda = -0.05,
    rho_theta_lambda = 0.64,
)

p = params
p

PCAffineParams(r0=-0.0042, theta_0=0.0344, lambda_0=0.0744, mu_r=0.0, g_T=array([0., 0., 1.]), kappa_r=0.3437, kappa_theta=0.0085, kappa_lambda=0.2816, sigma_r=0.005, sigma_theta=0.0157, sigma_lambda=0.12, theta_hat_t=0.035, lambda_hat_t=0.1287, rho_r_theta=0.6, rho_r_lambda=-0.05, rho_theta_lambda=0.64)

In [132]:
pca_model = PCAffineModel(yields_data)
# pca_model.fit(eigenvalues, maturities)
# kappaMatrix = pca_model.makeKappaMatrix( p.kappa_lambda, p.sigma_theta, p.kappa_theta, p.kappa_r )
# AMatrix = pca_model.makeAMatrix(p, 10.)
# BMatrix = pca_model.makeBMatrix(params, 10.)

pca_model.fit()

              SVENY02       SVENY05       SVENY10
SVENY02  4.636868e-07  4.145176e-07  3.274264e-07
SVENY05  4.145176e-07  4.616632e-07  4.019395e-07
SVENY10  3.274264e-07  4.019395e-07  4.219698e-07


In [126]:
AMatrix, BMatrix

(53.196616378193, array([-0.23281066, -6.94254407, -2.81594116]))

In [31]:
eigenvalues, eigenvectors = LA.eig(kappaMatrix)

In [30]:
eigenvalues

array([0.3437, 0.0085, 0.2816])

In [33]:
eigenvectors @ np.diag(eigenvalues) @ LA.inv(eigenvectors)

array([[ 2.81600000e-01,  0.00000000e+00,  0.00000000e+00],
       [-1.57000000e-02,  8.50000000e-03,  0.00000000e+00],
       [-4.08763396e-18, -3.43700000e-01,  3.43700000e-01]])